In [70]:
from pathlib import Path
data_path = Path("K:\ke\sta\data\SpeechEEG\Cocktail Party")
import scipy.io as sio
import numpy as np
import xarray as xr
import os
from signalstore import UnitOfWorkProvider
from mongomock import MongoClient
#from pymongo import MongoClient
from fsspec.implementations.local import LocalFileSystem
from fsspec import get_mapper
from fsspec.implementations.dirfs import DirFileSystem
from tempfile import TemporaryDirectory
import fsspec

# Cocktail Party Experiment

## EEG Files
eegData: EEG Data, Time Locked to the onset of the speech stimulus.   
Format: Channels (128) x Time Points

mastoids: Mastoid Channels, Time Locked to the onset of the speech stimulus. 
Format: Channels (Left=1 Right=2) x Time Points

fs: Sampling Rate 

EEG data is unfiltered, unreferenced and sampled at 128Hz

## Experiment Information
Subjects 1-17 were instructed to attend to 'Twenty Thousand Leagues Under the Sea' (20000), played in the left ear
Subjects 18-33 were instructed to attend to 'Journey to the Centre of the Earth' (Journey), played in the right ear

## Behavioral Data
score: Comprehension question scores for attended and unattended stories.
Format: Subjects x Run x Story (1=Attended, 2=Unattended)

## Stimuli Data Files

wordVec = List of all the content words for a given trial
onset_time = Onset time of the word in the corresponding cell of 'wordVec' (given in seconds)
offset_time = Offset time of the word in the corresponding cell of 'wordVec' (given in seconds)
sentence_boundaries = Time of sentence close (in seconds)

In [103]:
def load_eeg_data(data_path):
    eeg_dir = data_path / "EEG"
    subjects = [sdir for sdir in os.listdir(eeg_dir) if os.path.isdir(eeg_dir / sdir)]
    behavior_path = data_path / "Behavioural Data" / "Comprehension Scores.mat"
    behavior_data = sio.loadmat(behavior_path)
    behavior_scores = behavior_data['score']
    for sub_n, subject in enumerate(subjects):
        sub_n += 1
        subject_dir = eeg_dir / subject
        # list run mat files
        runs = os.listdir(subject_dir)
        for run_n, run in enumerate(runs):
            run_n += 1
            mfile = sio.loadmat(subject_dir / run)
            eeg_data = np.array(mfile['eegData'])
            print(eeg_data.shape)
            mastoid_data = mfile['mastoids']
            fs = mfile['fs']
            max_time = eeg_data.shape[0] / fs
            time_steps = np.linspace(0, max_time, eeg_data.shape[0]).squeeze()
            if sub_n <= 17:
                attending_direction = 'left'
                attending_story = 'Twenty Thousand Leagues Under the Sea'
            else:
                attending_direction = 'right'
                attending_story = 'Journey to the Center of the Earth'
            behavior_score = behavior_scores[sub_n-1, run_n-1]
            eeg_xarray = xr.DataArray(
                data=eeg_data,
                dims=['time', 'channel'],
                coords={
                    'time': time_steps,
                },
                attrs={
                    'schema_ref': 'eeg_signal',
                    'data_name': f'subject_{sub_n}_run_{run_n}_channels',
                    'subject': f'subject_{sub_n}',
                    'session_ref': {'schema_ref': 'session_record', 'data_name': f'session_{run_n}'},
                    'sampling_frequency': fs,
                    'attending_direction': attending_direction,
                    'attending_story': attending_story,
                    'attend_score': float(behavior_score[0]),
                    'nonattend_score': float(behavior_score[1])
                }
            )
            mastoid_xarray = xr.DataArray(
                data=mastoid_data,
                dims=['time', 'channel'],
                coords={
                    'time': time_steps,
                    'channel': ['left', 'right']
                },
                attrs={
                    'schema_ref': 'eeg_signal',
                    'data_name': f'subject_{sub_n}_run_{run_n}_mastoid',
                    'subject': f'subject_{sub_n}',
                    'session_ref': {'schema_ref': 'session_record', 'data_name': f'session_{run_n}'},
                    'sampling_frequency': fs,
                    'attending_direction': attending_direction,
                    'attending_story': attending_story,
                    'attend_score': float(behavior_score[0]),
                    'nonattend_score': float(behavior_score[1])
                }
            )
            yield eeg_xarray, mastoid_xarray

In [78]:
def load_stimuli(data_path):
    stimuli_dir = data_path / "Stimuli"
    envelopes_2000_dir = stimuli_dir / "Envelopes" / "2000"
    envelopes_2000_files = os.listdir(envelopes_2000_dir)
    envelopes_journey_dir = stimuli_dir / "Envelopes" / "Journey"
    envelopes_journey_files = os.listdir(envelopes_journey_dir)
    text_2000_dir = stimuli_dir / "Text" / "2000"
    text_2000_files = os.listdir(text_2000_dir)
    text_journey_dir = stimuli_dir / "Text" / "Journey"
    text_journey_files = os.listdir(text_journey_dir)
    assert len(envelopes_2000_files) == len(text_2000_files)
    assert len(envelopes_journey_files) == len(text_journey_files)
    assert len(envelopes_2000_files) == len(envelopes_journey_files)
    n_runs = len(envelopes_2000_files)
    for run in range(n_runs):
        stimuli_record = {
            'schema_ref': 'stimuli_record',
            'data_name': f'session_{run}',
            'left_wordvec_ref': {'schema_ref': 'wordvec', 'data_name': f'2000_run_{run}'},
            'left_offet_time_ref': {'schema_ref': 'offset_times', 'data_name': f'2000_run_{run}'},
            'left_onset_time_ref': {'schema_ref': 'onset_times', 'data_name': f'2000_run_{run}'},
            'left_sentence_boundaries_ref': {'schema_ref': 'sentence_boundaries', 'data_name': f'2000_run_{run}'},
            'left_envelope_ref': {'schema_ref': 'envelope', 'data_name': f'2000_run_{run}'},
            'right_wordvec_ref': {'schema_ref': 'wordvec', 'data_name': f'journey_run_{run}'},
            'right_offet_time_ref': {'schema_ref': 'offset_times', 'data_name': f'journey_run_{run}'},
            'right_onset_time_ref': {'schema_ref': 'onset_times', 'data_name': f'journey_run_{run}'},
            'right_sentence_boundaries_ref': {'schema_ref': 'sentence_boundaries', 'data_name': f'journey_run_{run}'},
            'right_envelope_ref': {'schema_ref': 'envelope', 'data_name': f'journey_run_{run}'}
        }
        # text contains keys: 'offset_time', 'onset_time', 'sentence_boundaries', 'wordVec'
        text_2000 = sio.loadmat(text_2000_dir / text_2000_files[run])
        wordvec_2000 = xr.Dataset(
            {
                'wordvec': (['time'], text_2000['wordVec']),
                'onset_time': (['time'], text_2000['onset_time'].flatten()),
                'offset_time': (['time'], text_2000['offset_time'].flatten())
            },

        )
        text_journey = sio.loadmat(text_journey_dir / text_journey_files[run])
        # envelopes contains keys: 'envelope', 'fsEnv', 'origLength'
        envelopes_2000 = sio.loadmat(envelopes_2000_dir / envelopes_2000_files[run])
        envelopes_journey = sio.loadmat(envelopes_journey_dir / envelopes_journey_files[run])
        left_wordvec = xr.DataArray(
            wordvec_2000['wordvec'],
            dims = ['word_number'],
            attrs={
                'schema_ref': 'wordvec',
                'data_name': f'2000_run_{run}',
                'story': 'Twenty Thousand Leagues Under the Sea',
                'stimuli_record_ref': {'schema_ref': 'stimuli_record', 'data_name': f'session_{run}'}
            }
        )
        right_wordvec = xr.DataArray(
            text_journey['wordVec'],
            dims = ['word_number'],
            attrs={
                'schema_ref': 'wordvec',
                'data_name': f'journey_run_{run}',
                'story': 'Journey to the Center of the Earth',
                'stimuli_record_ref': {'schema_ref': 'stimuli_record', 'data_name': f'session_{run}'}
            }
        )
        left_onset_time = xr.DataArray(
            text_2000['onset_time'],
            dims = ['word_number'],
            attrs={
                'schema_ref': 'onset_times',
                'data_name': f'2000_run_{run}',
                'story': 'Twenty Thousand Leagues Under the Sea',
                'stimuli_record_ref': {'schema_ref': 'stimuli_record', 'data_name': f'session_{run}'}
            }
        )
        right_onset_time = xr.DataArray(
            text_journey['onset_time'],
            dims = ['word_number'],
            attrs={
                'schema_ref': 'onset_times',
                'data_name': f'journey_run_{run}',
                'story': 'Journey to the Center of the Earth',
                'stimuli_record_ref': {'schema_ref': 'stimuli_record', 'data_name': f'session_{run}'}
            }
        )
        left_offset_time = xr.DataArray(
            text_2000['offset_time'],
            dims = ['word_number'],
            attrs={
                'schema_ref': 'offset_times',
                'data_name': f'2000_run_{run}',
                'story': 'Twenty Thousand Leagues Under the Sea',
                'stimuli_record_ref': {'schema_ref': 'stimuli_record', 'data_name': f'session_{run}'}
            }
        )
        right_offset_time = xr.DataArray(
            text_journey['offset_time'],
            dims = ['word_number'],
            attrs={
                'schema_ref': 'offset_times',
                'data_name': f'journey_run_{run}',
                'story': 'Journey to the Center of the Earth',
                'stimuli_record_ref': {'schema_ref': 'stimuli_record', 'data_name': f'session_{run}'}
            }
        )
        left_sentence_boundaries = xr.DataArray(
            text_2000['sentence_boundaries'],
            dims = ['sentence_number'],
            attrs={
                'schema_ref': 'sentence_boundaries',
                'data_name': f'2000_run_{run}',
                'story': 'Twenty Thousand Leagues Under the Sea',
                'stimuli_record_ref': {'schema_ref': 'stimuli_record', 'data_name': f'session_{run}'}
            }
        )
        right_sentence_boundaries = xr.DataArray(
            text_journey['sentence_boundaries'],
            dims = ['sentence_number'],
            attrs={
                'schema_ref': 'sentence_boundaries',
                'data_name': f'journey_run_{run}',
                'stimuli_record_ref': {'schema_ref': 'stimuli_record', 'data_name': f'session_{run}'}
            }
        )
        left_envelope = xr.DataArray(
            envelopes_2000['envelope'],
            dims = ['word_number'],
            attrs={
                'schema_ref': 'envelope',
                'data_name': f'2000_run_{run}',
                'sampling_frequency': envelopes_2000['fsEnv'],
                'origial_length': envelopes_2000['origLength'],
                'stimuli_record_ref': {'schema_ref': 'stimuli_record', 'data_name': f'session_{run}'}
            }
        )
        right_envelope = xr.DataArray(
            envelopes_journey['envelope'],
            dims = ['word_number'],
            attrs={
                'schema_ref': 'envelope',
                'data_name': f'journey_run_{run}',
                'sampling_frequency': envelopes_journey['fsEnv'],
                'origial_length': envelopes_journey['origLength'],
                'stimuli_record_ref': {'schema_ref': 'stimuli_record', 'data_name': f'session_{run}'}
            }
        )
        yield left_wordvec, right_wordvec, left_onset_time, right_onset_time, left_offset_time, right_offset_time, left_sentence_boundaries, right_sentence_boundaries, left_envelope, right_envelope, stimuli_record

In [ ]:
filesystem = LocalFileSystem()
tmp_dir = TemporaryDirectory()
print(tmp_dir.name)
tmp_dir_fs = DirFileSystem(
    tmp_dir.name,
    filesystem=filesystem
)
client = MongoClient()
memory_store = {}
uow_provider = UnitOfWorkProvider(
    mongo_client=client,
    filesystem=filesystem,
    memory_store=memory_store
)
import json
cwd = Path.cwd()
domain_models_path = cwd.parent / "Datasets\\cocktail_party\\models.json"
# load domain models json file
with open(domain_models_path) as f:
    domain_models = json.load(f)
for model in domain_models:
    print(f"Adding model {model['schema_name']} to domain_models store.")
    with uow_provider('cocktail-party') as uow:
        uow.domain_models.add(model)
        model = uow.domain_models.get(model['schema_name'])
        print(model)
for eeg_xarray, mastoid_xarray in load_eeg_data(data_path):
    with uow_provider('cocktail-party') as uow:
        uow.data.add(eeg_xarray)
        uow.data.add(mastoid_xarray)
for result in load_stimuli(data_path):
    for data in result:
        with uow_provider('cocktail-party') as uow:
            uow.data.add(data)

In [18]:
envelopes_path = "K:\ke\sta\data\SpeechEEG\Cocktail Party\Stimuli\Envelopes\\20000\\20000_1_env.mat"
matf = sio.loadmat(envelopes_path)
print(matf.keys())
print(matf['envelope'].shape)
print(matf['fsEnv'])
print(matf['origLength'])

dict_keys(['__header__', '__version__', '__globals__', 'envelope', 'fsEnv', 'origLength'])
(7680, 1)
[[128]]
[[7680]]
[]


In [43]:
text_path = "K:\ke\sta\data\SpeechEEG\Cocktail Party\Stimuli\Text\\20000\Run4.mat"
matf = sio.loadmat(text_path)
print(matf.keys())
def utf_array_to_str(arr):
    string = ""
    for word in arr:
        string += word[0][0] + " "
    return string
print(utf_array_to_str(matf['wordVec']))
print(matf['wordVec'].shape)
print(matf['onset_time'].shape)
print(matf['offset_time'].shape)
print(matf['sentence_boundaries'])

dict_keys(['__header__', '__version__', '__globals__', 'offset_time', 'onset_time', 'sentence_boundaries', 'wordVec'])
THEREFORE EVIDENT EXTRAORDINARY CREATURE COULD TRANSPORT ITSELF PLACE ANOTHER SURPRISING VELOCITY SEEING INTERVAL THREE DAYS BETWEEN TWO OBSERVATIONS SEPARATED DISTANCE SEVEN HUNDRED NAUTICAL LEAGUES FIFTEEN DAYS LATER TWO THOUSAND LEAGUES LAST PLACE SEEN HELVETIA COMPAGNIE NATIONALE SHANNON ROYAL MAIL STEAMSHIP COMPANY SAILING WINDWARD PART ATLANTIC BETWEEN UNITED STATES EUROPE EACH SIGNALLED MONSTER FORTY TWO DEGREES FIFTEEN MINUTES NORTH LATITUDE SIXTY DEGREES THIRTY FIVE MINUTES WEST LONGITUDE SHANNON HELVETIA SMALLER DIMENSIONS OBJECT MEASURED THREE HUNDRED FEET OVER MINIMUM LENGTH MAMMAL ESTIMATED THREE HUNDRED FIFTY FEET 
(84, 1)
(84, 1)
(84, 1)
[[18.673      47.773      60.53615474 60.        ]]


In [120]:
import re

def compute_unique_vocabulary(text_dir):
    full_paths = [text_dir / "20000" / f for f in os.listdir(text_dir / "20000")] \
                + [text_dir / "Journey" / f for f in os.listdir(text_dir / "Journey")]
    word_counts = {}
    word_lookup = {}
    for text_file in full_paths:
        matf = sio.loadmat(text_file)
        run = text_file.stem.replace("Run", "")
        source = text_file.parent.stem
        for word in matf['wordVec']:
            for word in word:
                word = str(word[0])
                if word in word_counts:
                    word_counts[word] += 1
                    word_lookup[word] += [(source, f'session_{run}')]
                else:
                    word_counts[word] = 1
                    word_lookup[word] = [(source, f'session_{run}')]
    return word_counts, word_lookup
vocab, lookup = compute_unique_vocabulary(data_path / "Stimuli" / "Text")
print(len(vocab))
print(lookup)

2171
{'CHAPTER': [('20000', 'session_1'), ('20000', 'session_16'), ('20000', 'session_29'), ('Journey', 'session_1'), ('Journey', 'session_13'), ('Journey', 'session_20')], 'FLOATING': [('20000', 'session_1'), ('20000', 'session_17'), ('20000', 'session_18')], 'REEF': [('20000', 'session_1'), ('20000', 'session_17'), ('20000', 'session_3'), ('20000', 'session_3')], 'YEAR': [('20000', 'session_1'), ('20000', 'session_3'), ('20000', 'session_8')], 'EIGHTEEN': [('20000', 'session_1'), ('20000', 'session_11'), ('20000', 'session_11'), ('20000', 'session_12'), ('20000', 'session_13'), ('20000', 'session_2'), ('20000', 'session_6'), ('20000', 'session_8'), ('20000', 'session_9'), ('Journey', 'session_6')], 'SIXTY': [('20000', 'session_1'), ('20000', 'session_11'), ('20000', 'session_12'), ('20000', 'session_13'), ('20000', 'session_2'), ('20000', 'session_22'), ('20000', 'session_4'), ('20000', 'session_5'), ('20000', 'session_8'), ('20000', 'session_9')], 'SIX': [('20000', 'session_1'), ('2